### 1. Instalação das bibliotecas necessárias

In [9]:
%pip install requests pandas plyer datetime IPython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### 2. Importação das bibliotecas

In [10]:
import pandas as pd
import requests
from plyer import notification
from IPython.display import display
from datetime import datetime
import sqlite3



### 3. Def de criação dos alertas

In [11]:
def alerta (nivel,base,etapa):
    """Exibe notificação em caso de falha de carregamento de base"""
    if 1 == nivel:
        notification.notify(
            title = 'Alerta Baixo',
            message = 'Falha do carregamento da base {} na etapa de {}'.format(base,etapa),
            app_name = 'Previsão - Brasil', timeout = 10
            )
    elif 2 == nivel:
        notification.notify(
            title = 'Alerta Medio',
            message = 'Falha do carregamento da base {} no(a) {}'.format(base,etapa),
            app_name = 'Previsão - Brasil', timeout = 10
            )
    elif 3 == nivel:
        notification.notify(
            title = 'Alerta Alto',
            message = 'Falha do carregamento da base {} no(a) {}'.format(base,etapa),
            app_name = 'Previsão - Brasil', timeout = 10
            )        
    else: 
        print('Erro sistemico. Por favor, tente novamente.')

### 4. Def para extrair dados da API e se necessário retornar mensagem de erro

In [12]:

def extracao_api (url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  
    elif response.status_code > 200 and response.status_code < 300:
        alerta(1,'para previsão','acesso')
    elif response.status_code >= 400 and response.status_code <500:
        alerta (2, 'para previsão', 'solicitação')
    elif response.status_code >= 500 and response.status_code <600:
        alerta (3, 'para previsão', 'servidor')
    else:
        alerta (1, 'para previsão', 'etapa desconhecida')

### 5.1. Trazendo os dados e trabalhando a  primeira tabela (estado)

In [13]:
url = "https://brasilapi.com.br/api/ibge/uf/v1"
data_json = extracao_api (url)
#estados_t = cabecalho(url,linhas=27)

regiao = [total_simples['regiao']['nome'] for total_simples in data_json]
nome = [total_simples['nome'] for total_simples in data_json]
sigla = [total_simples['sigla'] for total_simples in data_json]


total_simples = pd.DataFrame({
    'Nome': nome,
    'Sigla': sigla,
    'Regiao': regiao
})

sigla = 'RR'


estado = total_simples[total_simples['Sigla'] == sigla]
estado = estado.sort_values(by='Nome')


#Tabela 1: Estados e suas siglas
display(estado.head(len(estado)))


,Nome,Sigla,Regiao
3,Roraima,RR,Norte


### 5.2 Trazendo os dados e trabalhando a segunda tabela (Cidades do estado)

In [14]:
cidade = []
url = f'https://brasilapi.com.br/api/ibge/municipios/v1/{sigla}?providers=dados-abertos-br,gov,wikipedia'
data_json = extracao_api(url)

if data_json:
    cidade = [item['nome'] for item in data_json]

uf_e_cidades = pd.DataFrame({
    'Regiao': estado['Regiao'].values[0] if not estado.empty else None,
    'Estado': estado['Sigla'].values[0] if not estado.empty else None,
    'Cidade': cidade
})

uf = uf_e_cidades.sort_values(by='Cidade')


#Tabela 1: Estados e suas siglas do Centro-Oeste
display(uf.head(len(uf)))


,Regiao,Estado,Cidade
1,Norte,RR,ALTO ALEGRE
0,Norte,RR,AMAJARI
2,Norte,RR,BOA VISTA
3,Norte,RR,BONFIM
4,Norte,RR,CANTA
5,Norte,RR,CARACARAI
6,Norte,RR,CAROEBE
7,Norte,RR,IRACEMA
8,Norte,RR,MUCAJAI
9,Norte,RR,NORMANDIA


### 5.3 Trazendo os dados e trabalhando a terceira tabela (códigos CPTEC)

In [15]:
tabela_co = []
for cidade in uf['Cidade']:
    url = f'https://brasilapi.com.br/api/cptec/v1/cidade/{cidade}'
    data_json = extracao_api(url)
    if data_json:
        for item in data_json:
            if item['estado'] == sigla:  # Filtrar para cidades apenas do estado selecionado (evitar homonimas)
                df = pd.DataFrame({
                    'Estado': [item['estado']],
                    'Cidade': [item['nome']],
                    'Codigo': [item['id']]
                })
                tabela_co.append(df)
        
    # Combinar todos os DataFrames em um único DataFrame
total_co = pd.concat(tabela_co, ignore_index=True)
    # Exibir o DataFrame combinado com códigos
display(total_co.head(len(total_co)))

#Em testes, identificamos que as bases não estão atualizadas, apesar de ser a mesma API. Deste modo, algumas cidades são exibidas no quadro geral, mas não estão disponíveis na CPTEC. 
# Teste com estado do MS gerou divergência com perda de 3 itens



,Estado,Cidade,Codigo
0,RR,Alto Alegre,386
1,RR,Amajari,428
2,RR,Boa Vista,223
3,RR,Bonfim,940
4,RR,Cantá,1267
5,RR,Caracaraí,1312
6,RR,Caroebe,1371
7,RR,Iracema,2464
8,RR,Mucajaí,3396
9,RR,Normandia,3468


### 6 - RELACIONANDO TABELAS (Previsão meteorológica)

In [16]:

cid_temp = []

for codigos in total_co['Codigo']:
    url = f'https://brasilapi.com.br/api/cptec/v1/clima/previsao/{codigos}'
    data_json = extracao_api(url)

    if data_json:
        estado = data_json['estado']
        cidade = data_json['cidade']

        for clima in data_json['clima']:
             # Converter a data
            data_formatada = datetime.strptime(clima['data'], "%Y-%m-%d").strftime("%d/%m/%Y")
            df = pd.DataFrame({
                'Estado': [estado],
                'Cidade': [cidade],
                'Codigo': [codigos],
                'Data': [data_formatada],
                'Minima': [clima['min']],
                'Maxima': [clima['max']],
            })
            cid_temp.append(df)
    # Combinar todos os DataFrames em um único DataFrame
previsao = pd.concat(cid_temp, ignore_index=True)
    # Exibir o DataFrame combinado com códigos
display(previsao.head(len(previsao)))


,Estado,Cidade,Codigo,Data,Minima,Maxima
0,RR,Alto Alegre,386,04/07/2024,24,29
1,RR,Amajari,428,04/07/2024,23,29
2,RR,Boa Vista,223,04/07/2024,26,31
3,RR,Bonfim,940,04/07/2024,25,31
4,RR,Cantá,1267,04/07/2024,25,31
5,RR,Caracaraí,1312,04/07/2024,24,33
6,RR,Caroebe,1371,04/07/2024,23,30
7,RR,Iracema,2464,04/07/2024,23,32
8,RR,Mucajaí,3396,04/07/2024,24,32
9,RR,Normandia,3468,04/07/2024,24,30


### 7 - #def para carregamento de Dados no banco de dados

In [17]:
def carregar_dados_para_db(df, db_name="dados_ibge_cptec.db", table_name="previsao"):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    print(f"Dados carregados para {db_name} na tabela {table_name}")


### 8 - Chamada da função para carregamento no db da tabela


In [18]:
carregar_dados_para_db(previsao)


Dados carregados para dados_ibge_cptec.db na tabela previsao


### 9 - #DEF para visualizar dados usando queries


In [19]:
def executar_query(query, db_name="dados_ibge_cptec.db"):
    conn = sqlite3.connect(db_name)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

### 10 - Chamada da função para carregamento no db da tabela

In [21]:
query_previsao = "SELECT * FROM previsao"
df_previsao_query = executar_query(query_previsao)
print(df_previsao_query.head(len(df_previsao_query)))

   Estado              Cidade  Codigo        Data  Minima  Maxima
0      RR         Alto Alegre     386  04/07/2024      24      29
1      RR             Amajari     428  04/07/2024      23      29
2      RR           Boa Vista     223  04/07/2024      26      31
3      RR              Bonfim     940  04/07/2024      25      31
4      RR               Cantá    1267  04/07/2024      25      31
5      RR           Caracaraí    1312  04/07/2024      24      33
6      RR             Caroebe    1371  04/07/2024      23      30
7      RR             Iracema    2464  04/07/2024      23      32
8      RR             Mucajaí    3396  04/07/2024      24      32
9      RR           Normandia    3468  04/07/2024      24      30
10     RR           Pacaraima    3686  04/07/2024      20      28
11     RR        Rorainópolis    4449  04/07/2024      24      32
12     RR  São João da Baliza    4854  04/07/2024      24      31
13     RR            São Luiz    4983  04/07/2024      24      31
14     RR 

### 11 - Exemplo para erro na extração da base

In [22]:
url = f'https://brasilapi.com.br/api/cptec/v1/clima/previsao/'
data_json = extracao_api(url)